In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from unsloth import FastLanguageModel
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 512
lora_rank = 32 


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "peytonnash/piqa_10",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    fast_inference = True,
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 1
)

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA L4. Max memory: 21.951 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Your GPU cannot handle sequence lengths of 256 due to limited GPU memory.
Unsloth: Your GPU can only handle approximately the maximum sequence length of 256.
Unsloth: vLLM loading unsloth/meta-llama-3.1-8b-instruct-unsloth-bnb-4bit with actual GPU utilization = 4.81%
Unsloth: Your GPU has CUDA compute capability 8.9 with VRAM = 21.95 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 256. Num Sequences = 128.
Unsloth: vLLM's KV Cache can use up to 0.0 GB. Also swap space = 0 GB.
INFO 03-13 14:25:03 config.py:549] 

RuntimeError: CUDA out of memory. Tried to allocate 28.00 MiB. GPU 0 has a total capacity of 21.95 GiB of which 21.88 MiB is free. Process 25238 has 18.36 GiB memory in use. Process 45753 has 1.25 GiB memory in use. Including non-PyTorch memory, this process has 2.30 GiB memory in use. Of the allocated memory 2.09 GiB is allocated by PyTorch, and 5.98 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [6]:
# Load and prep dataset
SYSTEM_PROMPT = """
You are a helpful AI answering multiple-choice science questions. and respond in the following format:
<reason>  
...
</reason>  
<answer>  
...
</answer>  
"""

text = tokenizer.apply_chat_template([
        {"role": "system", "content": SYSTEM_PROMPT},
    ], tokenize=False, add_generation_prompt=True)

print(text)

NameError: name 'tokenizer' is not defined

In [18]:
def format_question(question, choices):
    choice_text = "\n".join([f"{chr(65+i)}) {choice}" for i, choice in enumerate(choices)])
    return tokenizer.apply_chat_template([
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"{question}\n\n{choice_text}\n\nChoose the best answer."}
    ], tokenize=False, add_generation_prompt=True)


In [21]:
def get_prediction(prompt, max_seq_length=2048):
    """Generates an answer using the fine-tuned model."""
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_seq_length).to(model.device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_length=min(inputs.input_ids.shape[1] + 256, max_seq_length),  # Ensure length limit
            temperature=0.7,
            top_p=0.9,
            do_sample=True
        )

    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

In [ ]:
from datasets import load_dataset
import re
from tqdm import tqdm  # Import tqdm for progress bar

arc_dataset = load_dataset("ai2_arc", "ARC-Challenge")
arc_test = arc_dataset["test"]
predictions = []

# Initialize counters for accuracy calculation
total_questions = 0
correct_predictions = 0

# Use tqdm to show progress bar
for example in tqdm(arc_test, desc="Processing ARC-Challenge Questions", unit="question"):
    formatted_question = format_question(example["question"], example["choices"]["text"])
    model_output = get_prediction(formatted_question)
    
    # Extract the letter choice from the model output (A, B, C, or D)
    pattern = r"<answer>\s*([A-D])\)"
    match = re.search(pattern, model_output)
    extracted_letter = match.group(1) if match else "Unknown"
    
    # Update counters
    total_questions += 1
    if extracted_letter == example["answerKey"]:
        correct_predictions += 1
    
    # Calculate current accuracy
    current_accuracy = (correct_predictions / total_questions) * 100
    
    predictions.append({
        "id": example["id"],
        "predicted": model_output,
        "extracted_answer": extracted_letter,
        "answerKey": example["answerKey"],
        "is_correct": extracted_letter == example["answerKey"]
    })
    
    print(f"Prediction: {extracted_letter}, Correct: {example['answerKey']}, Running Accuracy: {current_accuracy:.2f}%")

# Final accuracy
final_accuracy = (correct_predictions / len(arc_test)) * 100
print(f"\nFinal Accuracy: {final_accuracy:.2f}% ({correct_predictions}/{len(arc_test)})")

Processing ARC-Challenge Questions:   0%|          | 1/1172 [00:06<2:13:55,  6.86s/question]

Prediction: C, Correct: C, Running Accuracy: 100.00%


Processing ARC-Challenge Questions:   0%|          | 2/1172 [00:14<2:22:01,  7.28s/question]

Prediction: B, Correct: B, Running Accuracy: 100.00%


Processing ARC-Challenge Questions:   0%|          | 3/1172 [00:23<2:37:02,  8.06s/question]

...

Prediction: C, Correct: D, Running Accuracy: 64.67%


Processing ARC-Challenge Questions:  91%|█████████▏| 1071/1172 [2:35:06<13:32,  8.04s/question]

Prediction: Unknown, Correct: D, Running Accuracy: 64.61%


Processing ARC-Challenge Questions:  91%|█████████▏| 1072/1172 [2:35:14<13:10,  7.91s/question]

Prediction: A, Correct: A, Running Accuracy: 64.65%


In [ ]:
from sklearn.metrics import accuracy_score

y_true = [p["answerKey"] for p in predictions]
y_pred = [p["extracted_answer"] for p in predictions]

accuracy = accuracy_score(y_true, y_pred)
print(f"ARC Benchmark Accuracy: {accuracy:.2%}")

In [ ]:
import pandas as pd

df = pd.DataFrame(predictions)

df.to_csv("model_predictions.csv", index=False)